# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

In [1]:
# importeren libs
import numpy as np
import pandas as pd
import datetime
from operator import attrgetter
from itertools import product
import itertools
import pickle
import copy

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

Het gegeven excelbestand wat de informatie vat, is op een aantal punten aangepast voordat het is geimporteerd. 
Het bestand heeft voor iedere skill een aparte kolom. Mocht de skill nodig zijn voor de gegeven opdracht, dan staat er een 'x' in deze kolom. Ditzelfde is gedaan met de skills van medewerkers, de dagen waarop de medewerkers werken en voor de kamers waarin de opdrachten moeten worden uitgevoerd. Voor de additional tasks zijn er een aantal opdracht die in iedere sprint moeten worden uitgevoerd. Deze zijn opgesplitst, voor iedere sprint een aparte lijn. Hierbij is ook een kolom toegevoegd met het aantal frequenties voor deze opdracht. Ook is er een kolom toegevoegd bij het tabblad 'additional tasks' waarin staat aangegeven met hoeveel personen het minimaal moet worden uitgevoerd.
Hieronder worden de verschillende tabbladen ingelezen en toegekent aan een dataframe.
Verdere bewerkingen op de data worden gedaan in dit document.


In [2]:
# importeren data
dfProjectTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Project tasks')
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Rooms')
dfHiring = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Hiring')

Het schoonmaken van de cel, hierdoor kunnen we waardes makkelijker worden teruggevonden dan NaN

In [3]:
# aanpassen van de data (NaN -> 0 en x -> 1)
dfProjectTasks = dfProjectTasks.fillna(0)
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfCrew = dfCrew.fillna(0)
dfRooms = dfRooms.fillna(1)

dfProjectTasks.replace("x", 1, inplace=True)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfCrew.replace("x", 1, inplace=True)


**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken, daarna een half uur pauze
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' kan voor ieder crewlid worden gedaan, geen beperking in aantal 
- crewleden moet te samen aan de skills komen voor de opdracht.
- Mensen die skill één niet hebben, hoeven deze niet te leren
- Een crewleden kunnen alleen aan eengesloten uren aan een project task werken. Dit mag niet worden doorbroken met een andere task. Met additional tasks mag dit wel. 
- iedereen moet tegelijk aan het project werken als ze op hetzelfde project staan.

Moet A volledig worden afgerond of maar 1 keer van het aantal frequenties? Moet volledig worden ingepland

We maken een soort kalender, startend op 6 januari 2020 en geen weekend dagen erin.
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- een kolom met dataframes met de volgende punten:
    - tijd per halfuur
    - persoon
    - aantal uren te gaan
    - hierin staan de projecten met de kamer
- Of het een werkdag is (T/F)
- In welke sprint de dag zich bevindt.

In [4]:
# het maken van een dataframe als kalender << deze moet worden aangepast naar de info van hierboven

data = pd.date_range("2020-01-06", periods=4*10*7, freq='D')

columns = ['Datum', 'Sprint', 'Weeknummer', 'DagRooster', 'Werkdag']

dfKalenderCrew = pd.DataFrame(columns=columns)
dfKalenderCrew['Datum'] = data
dfKalenderCrew = dfKalenderCrew.fillna(0)
dfKalenderCrew.replace("0", False, inplace=True)

dfKalenderCrew['Werkdag'] = dfKalenderCrew['Datum'].apply(lambda Datum: Datum.isoweekday() in range (1,6))

In [5]:
dfKalenderCrew = dfKalenderCrew[dfKalenderCrew['Werkdag']]
dfKalenderCrew = dfKalenderCrew.reset_index(drop=True)

In [6]:
dfKalenderRoom = pickle.loads(pickle.dumps(dfKalenderCrew)) # het maken van een kalender van de kamers, tot dit punt waren de df's hetzelfde

In [7]:
# een paar standaard variabele om de code leesbaar te maken.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

functie waarbij de lijst van ReqSkill moet worden ingevuld van de Task en de skills van een persoon. De functie geeft een boolean terug of deze taak door deze persoon kan worden gedaan.

voorbeeld: <br>
OpdrachtUitvoeren(dfProjectTasks.iloc[0,rangeSkills],dfCrew.iloc[0,rangeSkills]) 
<br>uitkomst: False

In [8]:
def OpdrachtUitvoerenPersoonBool(eisen, persoon):
    for i in rangeSkills:
        result = (int(eisen[i-1]) <= 
                  int(persoon[i-1]))
        if result == False:
            break
    return result

de functie hieronder voegt de skills van personen samen en geeft de gezamelijke skills terug en het surplus.

voorbeeld: PersonenSamenvoegen(dfCrew.iloc[0,rangeSkills],dfCrew.iloc[2,rangeSkills])
<br>
uitkomst: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

In [9]:
def PersonenSamenvoegen(persoon1, persoon2 = [0]*13, persoon3 = [0]*13, persoon4 = [0]*13, persoon5 = [0]*13, persoon6 = [0]*13):
    combinedSkill = []
    surplus = 0 
    for i in rangeSkills:
        combinedSkill.append(int(1) if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
        surplus = surplus + (persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] -1 
                         if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
    combinedSkill.append(surplus)
    return combinedSkill

In [10]:
# het selecteren van alleen de tasks van sprint 1 en het dataframe maken met de columnnames en het resetten van de index
dfProjectTasks = dfProjectTasks[dfProjectTasks['Task'] != 0]
dfSprint1 = pd.DataFrame(dfProjectTasks[dfProjectTasks['Task'].str.contains('S1')], 
                         columns= list(dfProjectTasks)).reset_index(drop=True)


In [11]:
# het maken van de kolom crew en het aanpassen van het typen naar object zodat er een list of array kan worden toegevoegd aan die kolom.
dfSprint1.loc[:,'Crew']= [20]*len(dfSprint1)
dfSprint1['Crew'] = dfSprint1['Crew'].astype(object)

In [12]:
# # het verzamelen van data van alle gezamelijke skills in staan tot aan 7 personen

# dfCrewCombined = []
# for i in range(0, len(dfCrew)): 
#     t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills])
#     t.append(i)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(1)
    
#     dfCrewCombined.append(t)
    
#     for j in range(0, len(dfCrew)): # combinaties met twee mensen
#         if i==j:
#             break
#         t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills])
#         t.append(i)
#         t.append(j)
#         t.append(20)
#         t.append(20)
#         t.append(20)
#         t.append(20)
#         t.append(2)
        
#         dfCrewCombined.append(t)

#         for k in range(0, len(dfCrew)):
#             if i==k or k==j:
#                 break
        
#             t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills], dfCrew.iloc[k,rangeSkills])
#             t.append(i)
#             t.append(j)
#             t.append(k)
#             t.append(20)
#             t.append(20)
#             t.append(20)
#             t.append(3)
            
#             dfCrewCombined.append(t)
            
#             for l in range(0, len(dfCrew)):
#                 if l == k or l == i or l == j :
#                     break
#                 t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                         dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills])
#                 t.append(i)
#                 t.append(j)
#                 t.append(k)
#                 t.append(l)
#                 t.append(20)
#                 t.append(20)
#                 t.append(4)
#                 dfCrewCombined.append(t)

#                 for m in range(0, len(dfCrew)):
#                     if m==l or m==k or m==j or m==i:
#                         break
#                     t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                             dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
#                                             dfCrew.iloc[m,rangeSkills])
#                     t.append(i)
#                     t.append(j)
#                     t.append(k)
#                     t.append(l)
#                     t.append(m)
#                     t.append(20)
#                     t.append(5)
#                     dfCrewCombined.append(t)
                    
#                     for n in range(0, len(dfCrew)):
#                         if n==m or n==l or n==k or n==j or n==i:
#                             break
#                         t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                                 dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
#                                                 dfCrew.iloc[m,rangeSkills], dfCrew.iloc[n,rangeSkills])
#                         t.append(i)
#                         t.append(j)
#                         t.append(k)
#                         t.append(l)
#                         t.append(m)
#                         t.append(n)
#                         t.append(6)
#                         dfCrewCombined.append(t)


                    
       

het dataframe dfCrewCombined is een dataframe met alle skills van de medewerkers gezamelijk. Zodra er bij een van de personen '20' staat, betekent dit dat er geen persoon voor die plek is toegevoegd. in het dataframe staan combinaties met twee personen tot zeven ('vo) personen.


In [13]:
# # het maken van het dataframe dfCrewCombined en het wegschrijven in csv.
# ar = np.array(dfCrewCombined, dtype=np.int16) 
# dfCrewCombined = pd.DataFrame(ar, columns= ['Skill1', 'Skill2', 'Skill3', 'Skill4', 'Skill5', 'Skill6', 'Skill7', 'Skill8', 
#                                 'Skill9', 'Skill10', 'Skill11', 'Skill12', 'Skill13', 'Surplus', 'persoon1', 'persoon2', 
#                                 'persoon3', 'persoon4', 'persoon5', 'persoon6', 'Aantalpersonen'])

# # creeren van de kolom 'sum' zodat we weten hoeveel skill de crew gecombineerd heeft
# dfCrewCombined['sum'] = dfCrewCombined.iloc[:,0:13].sum(axis = 1) 

# # sorteren op de kolommen sum en surplus (overschot aan skills) zodat er zo min mogelijk overbodige skills worden gebruikt 
# dfCrewCombined = dfCrewCombined.sort_values(['sum', 'Surplus'], ascending= [True, True])
# dfCrewCombined = dfCrewCombined.reset_index(drop=True)

# dfCrewCombined.to_csv('CrewCombined.csv', encoding='utf-8', index=False) # schrijven naar csv

In [14]:
dfCrewCombined = pd.read_csv('CrewCombined.csv') # lezen van csv zodra die in wdir staat.

In [15]:
class MogelijkeCombinaties:
    def __init__(self, dfMogelijkheden, aantalMogelijkheden):
        self.dfMogelijkheden = dfMogelijkheden
        self.aantalMogelijkheden = aantalMogelijkheden

In [16]:
class Rooster:
    def __init__(self, dfRooster):
        self.dfRooster = dfRooster

In [17]:
def Lijstmogelijkheden(crew, duration): # lijst met mogelijke data wordt teruggegeven zodra de lijn van de crew en de duur van de task wordt ingevuld
    lijstmogelijkheden = []
    for dag in werkdagen:
        if crew == dag.crew:
            if dag.urenTeGaan >= duration :
                lijstmogelijkheden.append(dag)
    if len(lijstmogelijkheden) == 0:
        lijstmogelijkheden.append(RoosterDag("geen medewerker",datetime.date(2100,1,1)))
                                  
    return lijstmogelijkheden

In [18]:
dfSprint1.loc[:,'Mogelijkheden']= [20]*len(dfSprint1)
dfSprint1.loc[:,'AantalMogelijkheden']= [20]*len(dfSprint1)
dfSprint1['Mogelijkheden'] = dfSprint1['Mogelijkheden'].astype(object)
dfSprint1['AantalMogelijkheden'] = dfSprint1['AantalMogelijkheden'].astype(int)
dfSprint1['MaxReqCrew'].replace(0, 6, inplace=True)


In [19]:
def ZoekKamers(eisenKamers):
    lijstkamers = []
    for i in range(0, len(eisenKamers)):
        if list(eisenKamers)[i] == 1:
            if ("****" in list(dfSprint1)[14+i]):
                lijstkamers.append("assembly A")
                lijstkamers.append("assembly B")
            if ("***" in list(dfSprint1)[14+i] and not "****" in list(dfSprint1)[14+i]):
                lijstkamers.append("workshop A")
                lijstkamers.append("workshop B")
                lijstkamers.append("workshop C")
            if ("**" in list(dfSprint1)[14+i] and not "***" in list(dfSprint1)[14+i]):
                lijstkamers.append("workspace A")
                lijstkamers.append("workspace B")
                lijstkamers.append("workspace C")
                
            elif ("4" in list(dfSprint1)[14+i] or "5" in list(dfSprint1)[14+i]):
                lijstkamers.append("office A1")
                lijstkamers.append("office A2")
                lijstkamers.append("office A3")
                lijstkamers.append("office A4")
                lijstkamers.append("office A5")
                lijstkamers.append("office B1")
                lijstkamers.append("office B2")
                lijstkamers.append("office B3")
                lijstkamers.append("office B4")
                lijstkamers.append("office C1")
                lijstkamers.append("office C2")
                lijstkamers.append("office C3")
                lijstkamers.append("office C4")
                
            elif ("technical" in list(dfSprint1)[14+i]):
                lijstkamers.append("technical A")
                lijstkamers.append("technical B")
                
    return lijstkamers

In [20]:
def ZoekOpdrachten(project, MinMaxPersonen):
    crew = []
    for i in range(0, len(dfCrewCombined)):
        if (dfCrewCombined.Aantalpersonen[i] >= MinMaxPersonen[0] and dfCrewCombined.Aantalpersonen[i] <= MinMaxPersonen[1]):
            if (OpdrachtUitvoerenPersoonBool(project, dfCrewCombined.iloc[i,0:13])):
                crew.append(list(dfCrewCombined.iloc[i,14:20]))
    return crew
                        

In [21]:
dfSprint1['Voltooid'] = False
dfSprint1['Eis'] = ""
dfSprint1['Eis']=dfSprint1['Task'].apply(lambda Task: Task[:-1]+'A' if Task.endswith('B') else 'no')
dfSprint1['Moment Voltooid']= ""
dfSprint1['Dag Voltooid'] = ""

In [22]:
for i in range(0,len(dfSprint1)):
    a = ZoekKamers(dfSprint1.iloc[i,14:20])
    b = ZoekOpdrachten(dfSprint1.iloc[i,rangeSkills], dfSprint1.iloc[i,20:22]) 
    c = pd.DataFrame(data = [[a]*len(b),b])
    dfSprint1.iloc[i,26] = MogelijkeCombinaties(c.T, len(a) * len(b))
    dfSprint1.iloc[i,27] = (len(a) * len(b))/(dfSprint1["Frequency"][i]*dfSprint1["MinReqCrew"][i]) # >> is een optie, hoeft op zich niet, misschien wel makkelijker

In [23]:
dfSprint1 = dfSprint1.sort_values(['AantalMogelijkheden'], ascending= [True])
dfSprint1 = dfSprint1.reset_index(drop=True)

In [24]:
# het maken van een dataframe als kalender << deze moet worden aangepast naar de info van hierboven
tijd = pd.date_range("09:00", "17:30", freq="30min").time
tijd = np.delete(tijd, 13, axis=0)
tijd = np.delete(tijd, 6, axis=0)

columns = np.array(dfCrew['Crew members'])

dfRooster = pd.DataFrame(columns=columns)
dfRooster['Tijd'] = tijd
dfRooster = dfRooster.fillna(0)
dfRooster.replace("0", False, inplace=True)

# volgorde veranderen
cols = dfRooster.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfRooster = dfRooster[cols]

In [25]:
lijstkamers = dfRooms['Room'].iloc[0:10].tolist()

lijstkamers.append("office A1")
lijstkamers.append("office A2")
lijstkamers.append("office A3")
lijstkamers.append("office A4")
lijstkamers.append("office A5")
lijstkamers.append("office B1")
lijstkamers.append("office B2")
lijstkamers.append("office B3")
lijstkamers.append("office B4")
lijstkamers.append("office C1")
lijstkamers.append("office C2")
lijstkamers.append("office C3")
lijstkamers.append("office C4")

dfRoosterKamer = pd.DataFrame(columns=lijstkamers)
dfRoosterKamer['Tijd'] = tijd
dfRoosterKamer = dfRoosterKamer.fillna(0)
dfRoosterKamer.replace("0", False, inplace=True)

# volgorde veranderen
cols = dfRoosterKamer.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfRoosterKamer = dfRoosterKamer[cols]


In [26]:
a = [7]*(len(dfRooster.iloc[1, :])-1)
a.insert(0,'urenTeGaan')

b = dfRooster.columns.tolist()
new_row = pd.DataFrame(a).T
new_row.columns = b

dfRooster = pd.concat([new_row, dfRooster]).reset_index(drop = True)
StandaardRooster = dfRooster

In [27]:
for i in range(0,5): # het maken van de lege roosters voor week 1
    rooster = copy.deepcopy(StandaardRooster)
    rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
    dfKalenderCrew.DagRooster[i] = Rooster(rooster)
    dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))
    
for i in range(1,11): # het kopieren van de roosters van week 1 voor de rest van de weken in de sprint
    for j in range(0,5):
        dfKalenderCrew.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderCrew.DagRooster[j])
        dfKalenderRoom.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderRoom.DagRooster[j])

C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [28]:
def contains(sub, pri):
    M, N = len(pri), len(sub)
    i, LAST = 0, M-N+1
    while True:
        try:
            found = pri.index(sub[0], i, LAST) # find first elem in sub
        except ValueError:
            return False
        if pri[found:found+N] == sub:
            return [found, found+N-1]
        else:
            i = found+1

In [29]:
def CombinatieLijst(lijst1, lijst2):
    lijstfinal = []
    if ((len(lijst1) == 16) & (len(lijst2) == 16)):
        for i in range(0,len(lijst1)):
            if((lijst1[i]==0) & (lijst2[i] == 0)):
                lijstfinal.append(0)
            else:
                lijstfinal.append(1)
        return lijstfinal
    else:
        Exception("Lengte van de lijst klopt niet")
        print("Lengte van de lijst klopt niet")
        return
    

!TODO opdrachten over twee dagen

!TODO Opdracht volgorde veranderen. A moet eerst!

In [30]:
def InplannenOpdracht(taskdf, dagIndex, dfKalenderCrew, dfKalenderRoom): 
# task moet een rij van het dataframe dfSprint1 zijn en een dag (als index) van een rij in dfKalenderCrew (verschil ten opzichten van 2020-1-6)
#     print("Aanroep functie InplannenOpdracht met dagindex: " + str(dagIndex))
        
#     print("Er wordt ene kopie gemaakt van dfKalenderCrew")
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    
    for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist():
#         print("De crewlijst die wordt uitgeprobeerd: " + str(crewlijst))
        lijstVrijeUren = [0]*16
        controle = True
        dagIndexKopie = dagIndex
        
        freq = 1
        while (freq <= int(taskdf.Frequency)) & (taskdf.Voltooid == False):
#             print("bovenaan de while loop met: "+ taskdf.Task)
            if (taskdf.Eis != 'no') & (dagIndex == dagIndexKopie):
                eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
                lijstVrijeUren = [1]*int(eis["Moment Voltooid"]) + [0]*(16-int(eis["Moment Voltooid"]))
            for i in crewlijst:
#                 print("Kijken naar i: " + str(i))
                if (i == 20):
                    break
                if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)):
#                     print("De vergeleken waardes: "+ str(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1])+  " "+ str(task.Duration) )
#                     print("De controle van het aantal uur te gaan is False")
                    controle = False
                    break 
#                 print("de datum " + str(dfKalenderCrew.iloc[dagIndexKopie,:].Datum) + " voor persoon " +str(i))
#                 print("Het aantal uren te gaan voor persoon i " + str(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))
                lijstVrijeUren = CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
            duration = [0]*int(round(taskdf.Duration*2))

#             print("De lijstvrije uren ziet er als uit: " + str(lijstVrijeUren))
            if (contains(duration, lijstVrijeUren) != False) & controle:
                rijbegin , rijeind = contains(duration, lijstVrijeUren)

#                 print("het begin en het einde van de rij: "+ str(rijbegin +1) +" "+ str(rijeind+1))
                
                kamerGevonden = False
                for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]:
                    if (contains(duration, dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[1:].tolist()) != False):
#                         print("kamer gevonden: " + i)
                        dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1] = taskdf.Task + " freq: " + str(freq)
                        kamerGevonden = True
                        break
                    elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
#                         print("Geen kamer gevonden, controle False")
                        controle = False
                if (controle == True):
                    for i in crewlijst:
                        if i == 20:
                            break

                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task + " freq: " + str(freq) # + " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)

                    if (freq == int(taskdf.Frequency)):
#                         print("Voltooid " + taskdf.Task)
                        dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Moment Voltooid'] = rijeind +1
                        dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie
                        dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Voltooid'] = True

                        return [dfKalenderCrew, dfKalenderRoom]

#                     print("ingepland" + taskdf.Task + " " + str(freq))
                    freq += 1
            else:
#                 print("dagindex wordt verhoogd naar: " + str(dagIndexKopie+1))
                dagIndexKopie+=1

                controle = True
                lijstVrijeUren = [0]*16
                if (dagIndexKopie == 5):
#                     print("Een kopie wordt teruggezet van dfKalenderCrew")
                    dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                    dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                    break
    return [dfKalenderCrew, dfKalenderRoom]
    
    

In [31]:
dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfs = pickle.loads(pickle.dumps(dfSprint1))

In [32]:
# dfKalenderCrew = pickle.loads(pickle.dumps(dfk))
# dfSprint1 = pickle.loads(pickle.dumps(dfs))

In [33]:
for i in range(0,len(dfSprint1)):
    task = dfSprint1.iloc[i,:]
    dagIndex = 0
    # print(task.Task + " met: " + str(task.Voltooid))
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    if task.Eis != 'no':
        eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
        dfKalenderCrew, dfKalenderRoom = InplannenOpdracht(eis,dagIndex, dfKalenderCrew, dfKalenderRoom)
        eis = dfSprint1[dfSprint1.Task == task.Eis].iloc[0,:]
        print(eis.Task)
        dagIndex = int(eis['Dag Voltooid'])
    dfKalenderCrew, dfKalenderRoom = InplannenOpdracht(task,dagIndex, dfKalenderCrew, dfKalenderRoom)

C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


S1C3T5A
S1C1T1A
S1C2T3A
S1C2T2A
S1C3T2A


In [34]:
dfSprint1.iloc[:,[0,20,21,22,23,27,28,29,30,31]]

,Task,MinReqCrew,MaxReqCrew,Frequency,Duration,AantalMogelijkheden,Voltooid,Eis,Moment Voltooid,Dag Voltooid
0,S1C3T5B,1.0,2.0,2.0,2.0,28.000000,True,S1C3T5A,14,0
1,S1C2T3A,1.0,2.0,2.0,1.5,55.000000,True,no,6,1
2,S1C2T2A,1.0,2.0,4.0,2.0,57.500000,True,no,4,1
3,S1C3T5A,1.0,2.0,2.0,1.5,90.000000,True,no,6,0
4,S1C3T4,1.0,2.0,1.0,2.0,275.000000,True,no,4,0
5,S1C1T1B,3.0,6.0,4.0,2.0,565.000000,True,S1C1T1A,12,4
6,S1C3T1,3.0,4.0,1.0,3.0,620.000000,True,no,6,0
7,S1C2T1,4.0,5.0,1.0,2.0,810.000000,True,no,10,0
8,S1C2T3B,2.0,6.0,4.0,2.0,961.000000,True,S1C2T3A,8,3
9,S1C2T2B,3.0,6.0,2.0,2.0,1130.000000,True,S1C2T2A,8,2


In [35]:
dfKalenderCrew.iloc[0,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,0,2,0,0,1,1,1,1,2,1,5
1,09:00:00,S1C3T4 freq: 1,0,S1C3T1 freq: 1,S1C3T1 freq: 1,0,S1C3T5A freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C2T2A freq: 1,S1C3T1 freq: 1,S1C3T2A freq: 1,0
2,09:30:00,S1C3T4 freq: 1,0,S1C3T1 freq: 1,S1C3T1 freq: 1,0,S1C3T5A freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C2T2A freq: 1,S1C3T1 freq: 1,S1C3T2A freq: 1,0
3,10:00:00,S1C3T4 freq: 1,0,S1C3T1 freq: 1,S1C3T1 freq: 1,0,S1C3T5A freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C2T2A freq: 1,S1C3T1 freq: 1,S1C3T2A freq: 1,0
4,10:30:00,S1C3T4 freq: 1,0,S1C3T1 freq: 1,S1C3T1 freq: 1,0,S1C3T5A freq: 2,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C1T4 freq: 1,S1C2T2A freq: 1,S1C3T1 freq: 1,S1C3T2A freq: 1,0
5,11:00:00,S1C1T5 freq: 1,0,S1C3T1 freq: 1,S1C3T1 freq: 1,0,S1C3T5A freq: 2,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C2T2A freq: 2,S1C3T1 freq: 1,0,0
6,11:30:00,S1C1T5 freq: 1,0,S1C3T1 freq: 1,S1C3T1 freq: 1,0,S1C3T5A freq: 2,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C2T2A freq: 2,S1C3T1 freq: 1,0,0
7,12:30:00,S1C2T1 freq: 1,0,S1C1T3 freq: 1,S1C2T1 freq: 1,0,S1C3T5B freq: 1,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C2T2A freq: 2,S1C2T1 freq: 1,S1C1T3 freq: 1,S1C2T1 freq: 1
8,13:00:00,S1C2T1 freq: 1,0,S1C1T3 freq: 1,S1C2T1 freq: 1,0,S1C3T5B freq: 1,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C1T4 freq: 2,S1C2T2A freq: 2,S1C2T1 freq: 1,S1C1T3 freq: 1,S1C2T1 freq: 1
9,13:30:00,S1C2T1 freq: 1,0,S1C1T3 freq: 1,S1C2T1 freq: 1,0,S1C3T5B freq: 1,S1C1T4 freq: 3,S1C1T4 freq: 3,S1C1T4 freq: 3,S1C2T2A freq: 3,S1C2T1 freq: 1,S1C1T3 freq: 1,S1C2T1 freq: 1


In [36]:
dfKalenderCrew.iloc[1,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,1,1,1,7,0,1,1,1,1,1,3,1,1
1,09:00:00,S1C1T5 freq: 4,S1C2T4 freq: 1,S1C1T3 freq: 3,0,S1C2T3A freq: 1,S1C1T2 freq: 1,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C2T2A freq: 4,0,S1C1T3 freq: 3,S1C1T2 freq: 1
2,09:30:00,S1C1T5 freq: 4,S1C2T4 freq: 1,S1C1T3 freq: 3,0,S1C2T3A freq: 1,S1C1T2 freq: 1,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C2T2A freq: 4,0,S1C1T3 freq: 3,S1C1T2 freq: 1
3,10:00:00,S1C2T5 freq: 1,S1C2T4 freq: 1,S1C1T3 freq: 3,0,S1C2T3A freq: 1,S1C1T2 freq: 1,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C2T2A freq: 4,0,S1C1T3 freq: 3,S1C1T2 freq: 1
4,10:30:00,S1C2T5 freq: 1,S1C2T4 freq: 1,S1C1T3 freq: 3,0,S1C2T3A freq: 2,S1C1T2 freq: 1,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C2T2A freq: 4,0,S1C1T3 freq: 3,S1C1T2 freq: 1
5,11:00:00,S1C1T1A freq: 1,S1C2T4 freq: 2,S1C1T3 freq: 4,0,S1C2T3A freq: 2,S1C1T2 freq: 2,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T1A freq: 1,0,S1C1T3 freq: 4,S1C1T2 freq: 2
6,11:30:00,S1C1T1A freq: 1,S1C2T4 freq: 2,S1C1T3 freq: 4,0,S1C2T3A freq: 2,S1C1T2 freq: 2,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T1A freq: 1,0,S1C1T3 freq: 4,S1C1T2 freq: 2
7,12:30:00,S1C1T1A freq: 1,S1C2T4 freq: 2,S1C1T3 freq: 4,0,S1C2T3B freq: 1,S1C1T2 freq: 2,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T1A freq: 1,S1C2T3B freq: 1,S1C1T3 freq: 4,S1C1T2 freq: 2
8,13:00:00,S1C1T1A freq: 1,S1C2T4 freq: 2,S1C1T3 freq: 4,0,S1C2T3B freq: 1,S1C1T2 freq: 2,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T4 freq: 5,S1C1T1A freq: 1,S1C2T3B freq: 1,S1C1T3 freq: 4,S1C1T2 freq: 2
9,13:30:00,S1C1T1A freq: 2,S1C2T4 freq: 3,S1C1T3 freq: 5,0,S1C2T3B freq: 1,S1C1T2 freq: 3,S1C1T4 freq: 6,S1C1T4 freq: 6,S1C1T4 freq: 6,S1C1T1A freq: 2,S1C2T3B freq: 1,S1C1T3 freq: 5,S1C1T2 freq: 3


In [37]:
dfKalenderCrew.iloc[2,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,5,0,3,1,0,0,3,7,0,1,1
1,09:00:00,0,0,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4,0,0,S1C2T2B freq: 1,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4
2,09:30:00,0,0,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4,0,0,S1C2T2B freq: 1,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4
3,10:00:00,0,0,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4,0,0,S1C2T2B freq: 1,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4
4,10:30:00,0,0,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4,0,0,S1C2T2B freq: 1,0,0,S1C2T2B freq: 1,S1C1T2 freq: 4
5,11:00:00,0,0,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5,0,0,S1C2T2B freq: 2,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5
6,11:30:00,0,0,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5,0,0,S1C2T2B freq: 2,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5
7,12:30:00,0,0,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5,0,0,S1C2T2B freq: 2,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5
8,13:00:00,0,0,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5,0,0,S1C2T2B freq: 2,0,0,S1C2T2B freq: 2,S1C1T2 freq: 5
9,13:30:00,0,0,S1C1T3 freq: 6,0,0,S1C1T2 freq: 6,0,0,0,0,0,S1C1T3 freq: 6,S1C1T2 freq: 6


In [38]:
dfKalenderCrew.iloc[3,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,1,7,3,7,3,1,1,3,1,3,1,1,1
1,09:00:00,S1C1T1A freq: 3,0,S1C1T3 freq: 7,0,S1C2T3B freq: 3,S1C1T2 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T1A freq: 3,S1C2T3B freq: 3,S1C1T3 freq: 7,S1C1T2 freq: 7
2,09:30:00,S1C1T1A freq: 3,0,S1C1T3 freq: 7,0,S1C2T3B freq: 3,S1C1T2 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T1A freq: 3,S1C2T3B freq: 3,S1C1T3 freq: 7,S1C1T2 freq: 7
3,10:00:00,S1C1T1A freq: 3,0,S1C1T3 freq: 7,0,S1C2T3B freq: 3,S1C1T2 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T1A freq: 3,S1C2T3B freq: 3,S1C1T3 freq: 7,S1C1T2 freq: 7
4,10:30:00,S1C1T1A freq: 3,0,S1C1T3 freq: 7,0,S1C2T3B freq: 3,S1C1T2 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T4 freq: 7,S1C1T1A freq: 3,S1C2T3B freq: 3,S1C1T3 freq: 7,S1C1T2 freq: 7
5,11:00:00,S1C1T1A freq: 4,0,S1C1T3 freq: 8,0,S1C2T3B freq: 4,S1C1T2 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T1A freq: 4,S1C2T3B freq: 4,S1C1T3 freq: 8,S1C1T2 freq: 8
6,11:30:00,S1C1T1A freq: 4,0,S1C1T3 freq: 8,0,S1C2T3B freq: 4,S1C1T2 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T1A freq: 4,S1C2T3B freq: 4,S1C1T3 freq: 8,S1C1T2 freq: 8
7,12:30:00,S1C1T1A freq: 4,0,S1C1T3 freq: 8,0,S1C2T3B freq: 4,S1C1T2 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T1A freq: 4,S1C2T3B freq: 4,S1C1T3 freq: 8,S1C1T2 freq: 8
8,13:00:00,S1C1T1A freq: 4,0,S1C1T3 freq: 8,0,S1C2T3B freq: 4,S1C1T2 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T4 freq: 8,S1C1T1A freq: 4,S1C2T3B freq: 4,S1C1T3 freq: 8,S1C1T2 freq: 8
9,13:30:00,S1C1T1B freq: 1,0,0,0,0,S1C1T1B freq: 1,S1C3T3 freq: 1,0,S1C3T3 freq: 1,0,S1C3T2B freq: 1,S1C3T2B freq: 1,S1C1T1B freq: 1


In [39]:
dfKalenderCrew.iloc[4,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,1,0,7,0,0,1,1,7,1,0,0,7,1
1,09:00:00,S1C1T1B freq: 2,0,0,0,0,S1C1T1B freq: 2,S1C3T3 freq: 2,0,S1C3T3 freq: 2,0,0,0,S1C1T1B freq: 2
2,09:30:00,S1C1T1B freq: 2,0,0,0,0,S1C1T1B freq: 2,S1C3T3 freq: 2,0,S1C3T3 freq: 2,0,0,0,S1C1T1B freq: 2
3,10:00:00,S1C1T1B freq: 2,0,0,0,0,S1C1T1B freq: 2,S1C3T3 freq: 2,0,S1C3T3 freq: 2,0,0,0,S1C1T1B freq: 2
4,10:30:00,S1C1T1B freq: 2,0,0,0,0,S1C1T1B freq: 2,S1C3T3 freq: 2,0,S1C3T3 freq: 2,0,0,0,S1C1T1B freq: 2
5,11:00:00,S1C1T1B freq: 3,0,0,0,0,S1C1T1B freq: 3,S1C3T3 freq: 3,0,S1C3T3 freq: 3,0,0,0,S1C1T1B freq: 3
6,11:30:00,S1C1T1B freq: 3,0,0,0,0,S1C1T1B freq: 3,S1C3T3 freq: 3,0,S1C3T3 freq: 3,0,0,0,S1C1T1B freq: 3
7,12:30:00,S1C1T1B freq: 3,0,0,0,0,S1C1T1B freq: 3,S1C3T3 freq: 3,0,S1C3T3 freq: 3,0,0,0,S1C1T1B freq: 3
8,13:00:00,S1C1T1B freq: 3,0,0,0,0,S1C1T1B freq: 3,S1C3T3 freq: 3,0,S1C3T3 freq: 3,0,0,0,S1C1T1B freq: 3
9,13:30:00,S1C1T1B freq: 4,0,0,0,0,S1C1T1B freq: 4,S1C3T3 freq: 4,0,S1C3T3 freq: 4,0,0,0,S1C1T1B freq: 4


In [40]:
def EvaluateProgram(dfTasks, dfRoosterVoor, dfRoosterNa):
    urenTeGaan, totaal, som, count = [0,0,0,0]
    for i in range(0,5): # deze kan worden verhoogt zodra er meer dagen worden ingevuld.
        urenTeGaan += sum(dfRoosterVoor.iloc[i,:].DagRooster.dfRooster.iloc[0,:].tolist()[1:])
        totaal += sum(dfRoosterNa.iloc[i,:].DagRooster.dfRooster.iloc[0,:].tolist()[1:])
    for i in range(0, len(dfTasks)):
        som += dfTasks.loc[i, 'MinReqCrew'] * dfTasks.loc[i,'Frequency'] * dfTasks.loc[i,'Duration']
        if (dfTasks.Voltooid[i] == False):
            count += 1
    print("Het aantal uur dat nog over is: " + str(totaal))
    print("Van het totaal aantal uren: " + str(urenTeGaan))
    print("Het aantal uur dat moest worden ingepland: " + str(som))
    print("Het aantal opdrachten dat niet zijn ingepland: " + str(count))

In [41]:
EvaluateProgram(dfSprint1,dfk,dfKalenderCrew)

Het aantal uur dat nog over is: 114.0
Van het totaal aantal uren: 364
Het aantal uur dat moest worden ingepland: 250.0
Het aantal opdrachten dat niet zijn ingepland: 0


In [42]:
dfKalenderRoom.iloc[0,:].DagRooster.dfRooster

,Tijd,assembly A,assembly B,workshop A,workshop B,workshop C,workspace A,workspace B,workspace C,technical A,...,office A4,office A5,office B1,office B2,office B3,office B4,office C1,office C2,office C3,office C4
0,09:00:00,0,0,S1C3T2A freq: 1,S1C3T1 freq: 1,0,S1C1T4 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,09:30:00,0,0,S1C3T2A freq: 1,S1C3T1 freq: 1,0,S1C1T4 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10:00:00,0,0,S1C3T2A freq: 1,S1C3T1 freq: 1,0,S1C1T4 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10:30:00,0,0,S1C3T2A freq: 1,S1C3T1 freq: 1,0,S1C1T4 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11:00:00,0,0,S1C2T2A freq: 2,S1C3T1 freq: 1,0,S1C1T5 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11:30:00,0,0,S1C2T2A freq: 2,S1C3T1 freq: 1,0,S1C1T5 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,12:30:00,S1C2T1 freq: 1,0,S1C2T2A freq: 2,0,0,S1C1T3 freq: 1,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
7,13:00:00,S1C2T1 freq: 1,0,S1C2T2A freq: 2,0,0,S1C1T3 freq: 1,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
8,13:30:00,S1C2T1 freq: 1,0,S1C2T2A freq: 3,0,0,S1C1T3 freq: 1,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
9,14:00:00,S1C2T1 freq: 1,0,S1C2T2A freq: 3,0,0,S1C1T3 freq: 1,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0


!TODO 99% betrouwbaarheidsinterval

Het bedrijf werkt in sprints. Om precies te zijn: 4 sprints van elk 10 weken. De weken
2 t/m 8 van elke sprint zijn de zogeheten projectweken

In [43]:
# het kopieren van de roosters. Deze zijn voor dit gedeelte gelijk in week 2 tot en met 8. 
# Hierna moeten de additional tasks worden ingepland.

for i in range(0,7):
    for j in range(0,5):
        dfKalenderRoom.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderRoom.iloc[j,:].DagRooster.dfRooster))
        dfKalenderCrew.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderCrew.iloc[j,:].DagRooster.dfRooster))
        
for i in range(0,5): # leeg maken van de eerste week.
    dfKalenderCrew.DagRooster[i] = Rooster(copy.deepcopy(StandaardRooster))
    dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))

C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


!TODO de mensen die de prep uitvoeren, moeten ook de projecttasks en post doen.

Wanneer kan er met de prep check-up worden begonnen en wanneer kan de check-up worden gedaan?

In [44]:
task = dfSprint1[dfSprint1.Task == "S1C1T2"].iloc[0]

In [45]:
eis = dfSprint1[dfSprint1.Task == "S1C3T5A"].iloc[0,:]

In [46]:
int(eis['Dag Voltooid'])

0